# 3D Brain Tissue Segmentation Visualization

This notebook provides an interactive interface to visualize 3D brain tissue segmentations from your ablation experiments.

## Features:
- Visualize segmentations from experiment results
- Select specific brain tissue labels to display
- Compare predictions with ground truth
- Interactive 3D rendering with VTK


## Setup and Imports


In [ ]:
import os
import sys
import glob
import numpy as np
import SimpleITK as sitk
from pathlib import Path

# Add project to path
sys.path.insert(0, '/storage/homefs/ms23d081/mialab')

# Import visualization module
from mia_experiments.core.segmentation_visualizer import SegmentationVisualizer

print("✓ Imports successful")


✓ Imports successful


## Configuration

Set the path to your experiment results directory:


In [ ]:
# Path to experiment results
EXPERIMENT = "exp_00_baseline_none"
EXPERIMENT_DIR = "./ablation_experiments/" + EXPERIMENT + "/2025-11-09-14-17-13"

# Path to test data (for ground truth)
TEST_DATA_DIR = "./data/test"

# Check if directories exist
if os.path.exists(EXPERIMENT_DIR):
    print(f"✓ Experiment directory found: {EXPERIMENT_DIR}")
    seg_files = glob.glob(os.path.join(EXPERIMENT_DIR, "*_SEG*.mha"))
    print(f"  Found {len(seg_files)} segmentation files")
else:
    print(f"✗ Experiment directory not found: {EXPERIMENT_DIR}")

if os.path.exists(TEST_DATA_DIR):
    print(f"✓ Test data directory found: {TEST_DATA_DIR}")
else:
    print(f"✗ Test data directory not found: {TEST_DATA_DIR}")


✓ Experiment directory found: ./ablation_experiments/exp_00_baseline_none/2025-11-09-14-17-13
  Found 20 segmentation files
✓ Test data directory found: ./data/test


## Find Available Segmentations

List all available segmentation files:


In [ ]:
def find_segmentation_files(experiment_dir):
    """Find all segmentation files in experiment directory."""
    seg_files = glob.glob(os.path.join(experiment_dir, "*_SEG.mha"))
    seg_pp_files = glob.glob(os.path.join(experiment_dir, "*_SEG-PP.mha"))
    
    files_dict = {}
    for f in seg_files:
        subject_id = os.path.basename(f).split("_")[0]
        if subject_id not in files_dict:
            files_dict[subject_id] = {}
        files_dict[subject_id]['segmentation'] = f
    
    for f in seg_pp_files:
        subject_id = os.path.basename(f).split("_")[0]
        if subject_id not in files_dict:
            files_dict[subject_id] = {}
        files_dict[subject_id]['segmentation_pp'] = f
    
    return files_dict

# Find available files
available_files = find_segmentation_files(EXPERIMENT_DIR)

print("Available segmentations:")
print("=" * 60)
for subject_id, files in sorted(available_files.items()):
    print(f"\nSubject: {subject_id}")
    if 'segmentation' in files:
        print(f"  ✓ Segmentation: {os.path.basename(files['segmentation'])}")
    if 'segmentation_pp' in files:
        print(f"  ✓ Post-processed: {os.path.basename(files['segmentation_pp'])}")


Available segmentations:

Subject: 117122
  ✓ Segmentation: 117122_SEG.mha
  ✓ Post-processed: 117122_SEG-PP.mha

Subject: 118528
  ✓ Segmentation: 118528_SEG.mha
  ✓ Post-processed: 118528_SEG-PP.mha

Subject: 118730
  ✓ Segmentation: 118730_SEG.mha
  ✓ Post-processed: 118730_SEG-PP.mha

Subject: 118932
  ✓ Segmentation: 118932_SEG.mha
  ✓ Post-processed: 118932_SEG-PP.mha

Subject: 120111
  ✓ Segmentation: 120111_SEG.mha
  ✓ Post-processed: 120111_SEG-PP.mha

Subject: 122317
  ✓ Segmentation: 122317_SEG.mha
  ✓ Post-processed: 122317_SEG-PP.mha

Subject: 122620
  ✓ Segmentation: 122620_SEG.mha
  ✓ Post-processed: 122620_SEG-PP.mha

Subject: 123117
  ✓ Segmentation: 123117_SEG.mha
  ✓ Post-processed: 123117_SEG-PP.mha

Subject: 123925
  ✓ Segmentation: 123925_SEG.mha
  ✓ Post-processed: 123925_SEG-PP.mha

Subject: 124422
  ✓ Segmentation: 124422_SEG.mha
  ✓ Post-processed: 124422_SEG-PP.mha


## Visualize Single Segmentation

Visualize a segmentation file. You can specify:
- `subject_id`: Subject ID (e.g., '118528')
- `use_postprocessed`: Whether to use post-processed segmentation (SEG-PP.mha)
- `labels_to_show`: List of labels to visualize (1=WhiteMatter, 2=GreyMatter, 3=Hippocampus, 4=Amygdala, 5=Thalamus), or None for all


In [ ]:
""" # Configuration
SUBJECT_ID = "118528"  # Change this to visualize a different subject
USE_POSTPROCESSED = False  # Set to True to use SEG-PP.mha instead of SEG.mha
LABELS_TO_SHOW = None  # None = all labels, or [1, 2, 3] for specific labels

# Find the segmentation file
if SUBJECT_ID not in available_files:
    print(f"Error: Subject {SUBJECT_ID} not found in available files")
    print(f"Available subjects: {list(available_files.keys())}")
else:
    files = available_files[SUBJECT_ID]
    
    if USE_POSTPROCESSED and 'segmentation_pp' in files:
        seg_file = files['segmentation_pp']
        print(f"Using post-processed segmentation: {os.path.basename(seg_file)}")
    elif 'segmentation' in files:
        seg_file = files['segmentation']
        print(f"Using segmentation: {os.path.basename(seg_file)}")
    else:
        print(f"Error: No segmentation file found for subject {SUBJECT_ID}")
        seg_file = None
    
    if seg_file:
        # Load and inspect segmentation
        print(f"\nLoading: {seg_file}")
        image = sitk.ReadImage(seg_file)
        seg_array = sitk.GetArrayFromImage(image)
        spacing = image.GetSpacing()
        
        print(f"Shape: {seg_array.shape}")
        print(f"Spacing: {spacing} mm")
        print(f"Unique labels: {sorted(np.unique(seg_array))}")
        
        # Create visualizer
        visualizer = SegmentationVisualizer(spacing=spacing)
        
        # Visualize
        print("\nOpening 3D visualization window...")
        print("Close the window when done viewing.")
        visualizer.visualize(
            seg_array, 
            labels_to_show=LABELS_TO_SHOW,
            title=f"Subject {SUBJECT_ID} - Brain Tissue Segmentation"
        ) """


' # Configuration\nSUBJECT_ID = "118528"  # Change this to visualize a different subject\nUSE_POSTPROCESSED = False  # Set to True to use SEG-PP.mha instead of SEG.mha\nLABELS_TO_SHOW = None  # None = all labels, or [1, 2, 3] for specific labels\n\n# Find the segmentation file\nif SUBJECT_ID not in available_files:\n    print(f"Error: Subject {SUBJECT_ID} not found in available files")\n    print(f"Available subjects: {list(available_files.keys())}")\nelse:\n    files = available_files[SUBJECT_ID]\n    \n    if USE_POSTPROCESSED and \'segmentation_pp\' in files:\n        seg_file = files[\'segmentation_pp\']\n        print(f"Using post-processed segmentation: {os.path.basename(seg_file)}")\n    elif \'segmentation\' in files:\n        seg_file = files[\'segmentation\']\n        print(f"Using segmentation: {os.path.basename(seg_file)}")\n    else:\n        print(f"Error: No segmentation file found for subject {SUBJECT_ID}")\n        seg_file = None\n    \n    if seg_file:\n        # Loa

## Visualize Specific Labels Only

Visualize only specific brain tissues:


In [ ]:
""" # Label mapping:
# 1 = White Matter
# 2 = Grey Matter
# 3 = Hippocampus
# 4 = Amygdala
# 5 = Thalamus

SUBJECT_ID = "118528"
USE_POSTPROCESSED = False

# Show only specific labels (e.g., subcortical structures)
LABELS_TO_SHOW = [3, 4, 5]  # Hippocampus, Amygdala, Thalamus

if SUBJECT_ID in available_files:
    files = available_files[SUBJECT_ID]
    
    if USE_POSTPROCESSED and 'segmentation_pp' in files:
        seg_file = files['segmentation_pp']
    elif 'segmentation' in files:
        seg_file = files['segmentation']
    else:
        seg_file = None
    
    if seg_file:
        # Load segmentation
        image = sitk.ReadImage(seg_file)
        seg_array = sitk.GetArrayFromImage(image)
        spacing = image.GetSpacing()
        
        # Create visualizer
        visualizer = SegmentationVisualizer(spacing=spacing)
        
        label_names = {1: 'WhiteMatter', 2: 'GreyMatter', 3: 'Hippocampus', 
                      4: 'Amygdala', 5: 'Thalamus'}
        labels_str = ", ".join([label_names.get(l, str(l)) for l in LABELS_TO_SHOW])
        
        print(f"Visualizing labels: {labels_str}")
        print("Opening 3D visualization window...")
        visualizer.visualize(
            seg_array,
            labels_to_show=LABELS_TO_SHOW,
            title=f"Subject {SUBJECT_ID} - {labels_str}"
        )
 """

' # Label mapping:\n# 1 = White Matter\n# 2 = Grey Matter\n# 3 = Hippocampus\n# 4 = Amygdala\n# 5 = Thalamus\n\nSUBJECT_ID = "118528"\nUSE_POSTPROCESSED = False\n\n# Show only specific labels (e.g., subcortical structures)\nLABELS_TO_SHOW = [3, 4, 5]  # Hippocampus, Amygdala, Thalamus\n\nif SUBJECT_ID in available_files:\n    files = available_files[SUBJECT_ID]\n    \n    if USE_POSTPROCESSED and \'segmentation_pp\' in files:\n        seg_file = files[\'segmentation_pp\']\n    elif \'segmentation\' in files:\n        seg_file = files[\'segmentation\']\n    else:\n        seg_file = None\n    \n    if seg_file:\n        # Load segmentation\n        image = sitk.ReadImage(seg_file)\n        seg_array = sitk.GetArrayFromImage(image)\n        spacing = image.GetSpacing()\n        \n        # Create visualizer\n        visualizer = SegmentationVisualizer(spacing=spacing)\n        \n        label_names = {1: \'WhiteMatter\', 2: \'GreyMatter\', 3: \'Hippocampus\', \n                      4: \

## Compare Prediction with Ground Truth

Compare your segmentation prediction with the ground truth:


In [ ]:
def find_ground_truth(test_data_dir, subject_id):
    """Find ground truth segmentation file for a subject.
    
    Ground truth files are stored as NIfTI format (.nii.gz) with the pattern:
    labels_native.nii.gz
    """
    subject_dir = os.path.join(test_data_dir, subject_id)
    if not os.path.exists(subject_dir):
        return None
    
    # Look for ground truth file - NIfTI format
    # Primary: exact match for labels_native.nii.gz
    gt_file = os.path.join(subject_dir, "labels_native.nii.gz")
    if os.path.exists(gt_file):
        return gt_file
    
    # Fallback: pattern match for any labels_native*.nii.gz file
    gt_files = glob.glob(os.path.join(subject_dir, "*labels_native*.nii.gz"))
    if gt_files:
        return gt_files[0]
    
    return None


SUBJECT_ID = "118528"
LABELS_TO_SHOW = [3, 4, 5]  # None = all labels

# Find prediction file
if SUBJECT_ID in available_files:
    files = available_files[SUBJECT_ID]
    
    # find segmentations
    pred_file = files['segmentation']
    pred_file_pp = files['segmentation_pp']
    
    # Find ground truth file
    gt_file = find_ground_truth(TEST_DATA_DIR, SUBJECT_ID)
    
    if pred_file and pred_file_pp and gt_file:
        print(f"Prediction: {os.path.basename(pred_file)}")
        print(f"Prediction PP: {os.path.basename(pred_file_pp)}")
        print(f"Ground Truth: {os.path.basename(gt_file)}")
        
        # Load all three
        pred_image = sitk.ReadImage(pred_file)
        pred_array = sitk.GetArrayFromImage(pred_image)
        spacing = pred_image.GetSpacing()

        pred_image_pp = sitk.ReadImage(pred_file_pp)
        pred_array_pp = sitk.GetArrayFromImage(pred_image_pp)
        spacing_pp = pred_image_pp.GetSpacing()
        
        gt_image = sitk.ReadImage(gt_file)
        gt_array = sitk.GetArrayFromImage(gt_image)
        
        print(f"\nPrediction shape: {pred_array.shape}")
        print(f"\nPrediction PP shape: {pred_array_pp.shape}")
        print(f"Ground truth shape: {gt_array.shape}")
        print(f"Spacing: {spacing} mm")
        
        # Create visualizer
        visualizer = SegmentationVisualizer(spacing=spacing)
        
        # Prepare screenshot path
        screenshots_dir = Path("screenshots" + f"/{EXPERIMENT}")
        screenshots_dir.mkdir(exist_ok=True)

        screenshot_path = screenshots_dir / f"{SUBJECT_ID}_comparison.png"
        
        # Visualize comparison
        print("\nOpening comparison visualization...")
        visualizer.visualize_comparison(
            pred_array,
            pred_array_pp,
            gt_array,
            labels_to_show=LABELS_TO_SHOW,
            title=f"Subject {SUBJECT_ID} - Prediction vs Prediction PP vs Ground Truth",
            save_screenshot=screenshot_path
        )
    elif pred_file:
        print(f"Ground truth not found for subject {SUBJECT_ID}")
        print("Visualizing prediction only...")
        
        image = sitk.ReadImage(pred_file)
        seg_array = sitk.GetArrayFromImage(image)
        spacing = image.GetSpacing()
        
        visualizer = SegmentationVisualizer(spacing=spacing)
        visualizer.visualize(
            seg_array,
            labels_to_show=LABELS_TO_SHOW,
            title=f"Subject {SUBJECT_ID} - Prediction"
        )
    else:
        print(f"Error: No segmentation file found for subject {SUBJECT_ID}")


Prediction: 118528_SEG.mha
Prediction PP: 118528_SEG-PP.mha
Ground Truth: labels_native.nii.gz

Prediction shape: (181, 217, 181)

Prediction PP shape: (181, 217, 181)
Ground truth shape: (181, 217, 181)
Spacing: (1.0, 1.0, 1.0) mm

Opening comparison visualization...
Extracting prediction surfaces...
  Added Hippocampus (label 3)
  Added Amygdala (label 4)
  Added Thalamus (label 5)
Extracting prediction surfaces...
  Added Hippocampus (label 3)
  Added Amygdala (label 4)
  Added Thalamus (label 5)
Extracting ground truth surfaces...
  Added Hippocampus (label 3)
  Added Amygdala (label 4)
  Added Thalamus (label 5)

Displaying comparison view (3 prediction surfaces, 3 GT surfaces)
Controls:
  - Left click + drag: Rotate
  - Right click + drag: Zoom
  - Middle click + drag: Pan
  - Close window to exit

Screenshot saved to: screenshots\exp_00_baseline_none\118528_comparison.png


## Visualize from Custom File Path

Visualize any segmentation file by specifying its path directly:


In [ ]:
""" # Specify file path directly
CUSTOM_FILE = "./ablation_experiments/" + EXPERIMENT + "/2025-11-09-14-17-13/118528_SEG.mha"
LABELS_TO_SHOW = None  # None = all, or [1, 2, 3] for specific labels

if os.path.exists(CUSTOM_FILE):
    print(f"Loading: {CUSTOM_FILE}")
    
    # Load segmentation
    image = sitk.ReadImage(CUSTOM_FILE)
    seg_array = sitk.GetArrayFromImage(image)
    spacing = image.GetSpacing()
    
    print(f"Shape: {seg_array.shape}")
    print(f"Spacing: {spacing} mm")
    print(f"Unique labels: {sorted(np.unique(seg_array))}")
    
    # Create visualizer
    visualizer = SegmentationVisualizer(spacing=spacing)
    
    # Visualize
    print("\nOpening 3D visualization...")
    visualizer.visualize(
        seg_array,
        labels_to_show=LABELS_TO_SHOW,
        title=os.path.basename(CUSTOM_FILE)
    )
else:
    print(f"File not found: {CUSTOM_FILE}")
 """

' # Specify file path directly\nCUSTOM_FILE = "./ablation_experiments/" + EXPERIMENT + "/2025-11-09-14-17-13/118528_SEG.mha"\nLABELS_TO_SHOW = None  # None = all, or [1, 2, 3] for specific labels\n\nif os.path.exists(CUSTOM_FILE):\n    print(f"Loading: {CUSTOM_FILE}")\n    \n    # Load segmentation\n    image = sitk.ReadImage(CUSTOM_FILE)\n    seg_array = sitk.GetArrayFromImage(image)\n    spacing = image.GetSpacing()\n    \n    print(f"Shape: {seg_array.shape}")\n    print(f"Spacing: {spacing} mm")\n    print(f"Unique labels: {sorted(np.unique(seg_array))}")\n    \n    # Create visualizer\n    visualizer = SegmentationVisualizer(spacing=spacing)\n    \n    # Visualize\n    print("\nOpening 3D visualization...")\n    visualizer.visualize(\n        seg_array,\n        labels_to_show=LABELS_TO_SHOW,\n        title=os.path.basename(CUSTOM_FILE)\n    )\nelse:\n    print(f"File not found: {CUSTOM_FILE}")\n '

## Quick Visualization Helper Function

A convenient function to quickly visualize any segmentation:


In [ ]:
""" def quick_visualize(file_path, labels=None, title=None):
    """Quick visualization helper.
    
    Args:
        file_path: Path to .mha segmentation file
        labels: List of labels to show (None for all)
        title: Window title (default: filename)
    """
    if not os.path.exists(file_path):
        print(f"Error: File not found: {file_path}")
        return
    
    # Load
    image = sitk.ReadImage(file_path)
    seg_array = sitk.GetArrayFromImage(image)
    spacing = image.GetSpacing()
    
    # Visualize
    visualizer = SegmentationVisualizer(spacing=spacing)
    
    if title is None:
        title = os.path.basename(file_path)
    
    visualizer.visualize(seg_array, labels_to_show=labels, title=title)


# Example usage:
# quick_visualize("./ablation_experiments/" + EXPERIMENT + "/2025-11-09-14-17-13/118528_SEG.mha")
# quick_visualize("./ablation_experiments/" + EXPERIMENT + "/2025-11-09-14-17-13/118528_SEG.mha", labels=[3, 4, 5])
 """

SyntaxError: invalid syntax (1617299865.py, line 2)

## Notes

- **Label mapping**: 1=WhiteMatter, 2=GreyMatter, 3=Hippocampus, 4=Amygdala, 5=Thalamus
- **Interacting with 3D view**: 
  - Left mouse button: Rotate
  - Right mouse button: Zoom
  - Middle click + drag: Pan
  - Close window to continue
- **Post-processed vs regular**: SEG-PP.mha files have post-processing applied (e.g., hole filling, smoothing)
- The visualization window must be closed before running the next cell
